<h1><center>Sprint 7</center></h1>
<h2><center>Tasca 1: Algoritmes d’aprenentatge supervisat: Classificació.</center></h2>

In [ ]:
# Packages
from pathlib import Path
import numpy as np
import pandas as pd
from scipy import stats
import math
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
import itertools
import warnings

from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn import tree
from sklearn.preprocessing import MinMaxScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import f1_score, confusion_matrix, log_loss
from sklearn.metrics import precision_recall_fscore_support, accuracy_score, classification_report
from sklearn.model_selection import cross_validate

In [ ]:
# file paths
data_path = 'D:/Sistema_Solar/Python/itacademy/sprint07/data/'
data_path = Path(data_path)

- Exercici 1
Crea almenys dos models de classificació diferents per intentar predir el millor les classes de l'arxiu adjunt.



- Exercici 2
Compara els models de classificació utilitzant la precisió (accuracy), una matriu de confusió i d’altres mètriques més avançades.



- Exercici 3
Entrena’ls usant els diferents paràmetres que admeten per tal de millorar-ne la predicció.



- Exercici 4
Compara el seu rendiment fent servir l’aproximació traint/test o cross-validation.



- Exercici 5
Aplica algun procés d'enginyeria per millorar els resultats (normalització, estandardització, mostreig...)



**_Es realitzen els diferents exercicis de manera conjunta per a cada tipus de model aplicat_**

In [ ]:
# Apugem l'arxiu csv
file_name = 'wineData.txt'
file = data_path / file_name
wines = pd.read_csv(file, sep=',', header=None, encoding="ISO-8859-1")

# Afegim els noms de les columnes
colnames = ['Class', 'Alcohol', 'Malic', 'Ash', 'Alcalinity', 'Mg', 'Total phenols', 'Flavanoids', 'Nonflavanoids', 'Proanthocyanins', 'Color intensity', 'Hue', 'OD280/OD315', 'Proline']
wines.columns = colnames

In [ ]:
wines.info()

In [ ]:
print(wines.describe())

In [ ]:
wines_matrix = wines.corr()

# Eliminarem les correlacions entre iguals per millorar la visualització posterior
for i in range(len(wines_matrix.columns)):
    for j in range(len(wines_matrix.columns)):
        if i == j:
            wines_matrix.iloc[i][j] = np.nan

In [ ]:
fig, ax = plt.subplots(figsize=(18,10))         # Sample figsize in inches
#sns.heatmap(df1.iloc[:, 1:6:], annot=True, linewidths=.5, ax=ax)

sns.heatmap(wines_matrix, annot=True, linewidth=.5, ax=ax, cmap="PiYG", center=0)

De totes les variables de ls que disposem, algunes tenen poca relació amb la classe de vins i les descartarem com són: **Ash**, **Mg** i **Color intensity**

In [ ]:
wines.drop(['Ash', 'Mg', 'Color intensity'], axis=1, inplace=True)

In [ ]:
fig, ax = plt.subplots(4, 3, figsize=(23,18))
nrow = 0
ncol = 0

for col in wines.columns:
    sns.histplot(wines[col], ax=ax[nrow][ncol])
    ncol += 1
    if ncol == 3:
        nrow += 1
        ncol = 0

In [ ]:
# Dataframe on emmagatzarem els coeficients MSE i R2 per comparar els diferents models.

coef_df = pd.DataFrame(index = ['Accuracy', 'Precision', 'Recall', 'F1score'])

#### 1. Model d'arbre de decisió

In [ ]:
# Separem el dataframe en un conjunt per entrenar el model i un altre per aplicar-lo.

test_size_sel = 0.3

wines_train, wines_test = train_test_split(wines, test_size=test_size_sel, random_state=42)

In [ ]:
#col_sel = ['Alcohol', 'Malic', 'Alcalinity', 'Total phenols', 'Flavanoids', 'Nonflavanoids', 'Proanthocyanins', 'Hue', 'OD280/OD315', 'Proline']

x_col = wines.columns[1:]

x_train = wines_train[x_col]
y_train = wines_train['Class']

x_test = wines_test[x_col]
y_test = wines_test['Class']

Ajustem un primer model d'arbre de decisions sense definir cap paràmetre.

In [ ]:
# Model d'arbre de decisió categòric

dt = DecisionTreeClassifier(random_state=42)

dt.fit(x_train, y_train)
dt_depth = dt.get_depth()
print(f'Decision Tree depth: {dt_depth}')
y_pred = dt.predict(x_test)
y_pred_train = dt.predict(x_train)

In [ ]:
plt.figure(dpi=250)

cn = ['I', 'II', 'III'] 

plt.show(tree.plot_tree(dt,
               feature_names = x_col, 
               class_names=cn,
               filled = True))

El model té 4 nivells i es pot observar en el gràfic els diferents punts de bifurcació i com tots els nivells més baixos, les fulles, són totalment homogenis amb relació a les classes que contenen. Avaluem-lo amb diferents mètodes per veure el seu rendiment i si aquest es pot millorar.

In [ ]:
# funció per plotejar matrius de confusió

warnings.filterwarnings( "ignore")

def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=0)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" 
                 if cm[i, j] > thresh 
                 else "black")

    plt.tight_layout()
    plt.ylabel('Clase vertadera')
    plt.xlabel('Clase prevista')

In [ ]:
# Avaluació del model

# accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f'model accuracy: {np.round(accuracy, 2)}')

In [ ]:
# matriu de confusió

confusion_train = confusion_matrix(y_train, y_pred_train, labels=[1, 2, 3])
confusion_test = confusion_matrix(y_test, y_pred, labels=[1, 2, 3])

In [ ]:
#fig, ax = plt.subplots(1, 2, figsize=(23,18))
plt.figure(figsize=(5, 5))
sns.set(font_scale=1.3)
plot_confusion_matrix(confusion_train, 
                      classes=['I','II', 'III'],
                      normalize= False,  
                      title='Dades d\'entrenament')

In [ ]:
plt.figure(figsize=(5, 5))
sns.set(font_scale=1.3)
plot_confusion_matrix(confusion_test, 
                      classes=['I','II', 'III'],
                      normalize= False,  
                      title='Dades de testatge')

La matriu de confusió mostra com per a les dades de d'entrenament, el rendiment és perfecte, mentre que amb les dades per al testeig, 5 clases de vins no són correctament clasificades.

In [ ]:
# Més coeficients d'avaluació
precision = precision_recall_fscore_support(y_test, y_pred, average='macro')[0]
recall = precision_recall_fscore_support(y_test, y_pred, average='macro')[1]
f1score = f1_score(y_test, y_pred, average='micro')
print(f'Precision: {np.round(precision, 2)}')
print(f'Recall: {np.round(recall, 2)}')
print(f'F1 score: {np.round(f1score, 2)}')

In [ ]:
coef_df['DecTree_O'] = [np.round(accuracy,2), np.round(precision, 2), np.round(recall, 2), np.round(f1score, 2)]

In [ ]:
coef_df

El número de nivells de divisions d'aquest model era de 4, pero podem aconseguir diferents resultats si ajustem aquest paràmetre. Provarem diferents nivells màxims de 2 a 5.

In [ ]:
dpi_sel = [125, 210, 280]
for depth in range(2, 5):

    dt = DecisionTreeClassifier(max_depth=depth, random_state=42)

    dt.fit(x_train, y_train)
    dt_depth = dt.get_depth()
    y_pred = dt.predict(x_test)
    y_pred_train = dt.predict(x_train)


    plt.figure(dpi=dpi_sel[depth-2])

    cn = ['I', 'II', 'III'] 

    plt.show(tree.plot_tree(dt,
                   feature_names = x_col, 
                   class_names=cn,
                   filled = True))
    print(f'Decision Tree depth: {dt_depth}')
    accuracy = accuracy_score(y_test, y_pred)
    print(f'Accuracy: {np.round(accuracy, 2)}')
    
    precision = precision_recall_fscore_support(y_test, y_pred, average='macro')[0]
    recall = precision_recall_fscore_support(y_test, y_pred, average='macro')[1]
    f1score = f1_score(y_test, y_pred, average='micro')
    print(f'Precision: {np.round(precision, 2)}')
    print(f'Recall: {np.round(recall, 2)}')
    print(f'F1 score: {np.round(f1score, 2)}')
    print('----------------')

El model es comporta millor amb un maxim_depth de 3 o 4. Encara que algunes de les fulles finals ja no són totalment pures, la precisió és major. Si ens quedem sols amb 3 nivells, el nombre de variables que utilitza el model per anar separant les mostres són de 5 (OD280/OD315, HUE, ALCOHOL, FLAVANOIDS i PROLINE), en canvi, amb 4 nivells s'afegeix ALCALINITY tot i que sols serveix per variar el resultat en una mostra. Potser podem trobar més difer+ènciès en les mostres d'entrenament, cosa que podem comprovar amb el mètode de la validació creuada.

In [ ]:
for depth in range(2, 5):

    dt = DecisionTreeClassifier(max_depth=depth)

    dt.fit(x_train, y_train)
    dt_depth = dt.get_depth()
    y_pred = dt.predict(x_test)
    y_pred_train = dt.predict(x_train)

    scores = cross_validate(dt, x_train, y_train, cv=5, scoring='accuracy', return_train_score=True)
    
    print(f'Decision Tree depth: {dt_depth}')
    print(f'Entrenament: {scores["train_score"]}')
    print(f'Test       : {scores["test_score"]}')
    print('----------------')

Amb aquest mètode, veiem com els resultats amb les dades d'entrenament milloren amb un max_depth major. De fet, fent servir un max_depth de 2 és quan es pateix menys d'overfitting. Així i tot, s'ha vist que amb sols dos nivells de profunditat, s'obtenen moltes fulles impures i a més, el model potser prescindeix de massa variables perquè sols en gasta 3. Però en el cas de 4, el overfitting és major. Provarem de limitar el nombre mínim de mostres necessàries per a crear una altra fulla ajustant el paràmetre min_samples_leaf i emprant un max_depth de 3 o de 4.

In [ ]:
print('-------------------------------------------\n-------------- max_depth: 3 ---------------\n-------------------------------------------')
for leaf in range(1, 6):

    dt = DecisionTreeClassifier(max_depth=3, min_samples_leaf=leaf, random_state=42)

    dt.fit(x_train, y_train)
    dt_depth = dt.get_depth()
    y_pred = dt.predict(x_test)
    y_pred_train = dt.predict(x_train)

    print(f'min_samples_leaf: {leaf}')
    accuracy = accuracy_score(y_test, y_pred)
    print(f'Accuracy: {np.round(accuracy, 2)}')
    precision = precision_recall_fscore_support(y_test, y_pred, average='macro')[0]
    recall = precision_recall_fscore_support(y_test, y_pred, average='macro')[1]
    f1score = f1_score(y_test, y_pred, average='micro')
    print(f'Precision: {np.round(precision, 2)}')
    print(f'Recall: {np.round(recall, 2)}')
    print(f'F1 score: {np.round(f1score, 2)}')
    scores = cross_validate(dt, x_train, y_train, cv=5, scoring='accuracy', return_train_score=True)
    print(f'Entrenament: {scores["train_score"]}')
    print(f'Test       : {scores["test_score"]}')
    print('----------------')

In [ ]:
print('-------------------------------------------\n-------------- max_depth: 4 ---------------\n-------------------------------------------')
for leaf in range(1, 6):

    dt = DecisionTreeClassifier(max_depth=4, min_samples_leaf=leaf, random_state=42)

    dt.fit(x_train, y_train)
    dt_depth = dt.get_depth()
    y_pred = dt.predict(x_test)
    y_pred_train = dt.predict(x_train)

    print(f'min_samples_leaf: {leaf}')
    accuracy = accuracy_score(y_test, y_pred)
    print(f'Accuracy: {np.round(accuracy, 2)}')
    precision = precision_recall_fscore_support(y_test, y_pred, average='macro')[0]
    recall = precision_recall_fscore_support(y_test, y_pred, average='macro')[1]
    f1score = f1_score(y_test, y_pred, average='micro')
    print(f'Precision: {np.round(precision, 2)}')
    print(f'Recall: {np.round(recall, 2)}')
    print(f'F1 score: {np.round(f1score, 2)}')
    scores = cross_validate(dt, x_train, y_train, cv=5, scoring='accuracy', return_train_score=True)
    print(f'Entrenament: {scores["train_score"]}')
    print(f'Test       : {scores["test_score"]}')
    print('----------------')   

Si sols modifiquem el paràmetre max_depth, podem quedar-nos amb un valor de 3, però podem obtenir un millor rendiment si, amb un max_depth de 4, modifiquem el paràmetre min_samples_leaf amb un valor de 3 o 4. A més, que es redueix l'overfitting.

In [ ]:
dt = DecisionTreeClassifier(max_depth=4, min_samples_leaf=3, random_state=42)

dt.fit(x_train, y_train)
dt_depth = dt.get_depth()
y_pred = dt.predict(x_test)
y_pred_train = dt.predict(x_train)

plt.figure(dpi=250)

cn = ['I', 'II', 'III'] 

plt.show(tree.plot_tree(dt,
               feature_names = x_col, 
               class_names=cn,
               filled = True))

accuracy = accuracy_score(y_test, y_pred)
precision = precision_recall_fscore_support(y_test, y_pred, average='macro')[0]
recall = precision_recall_fscore_support(y_test, y_pred, average='macro')[1]
f1score = f1_score(y_test, y_pred, average='micro')

In [ ]:
coef_df['DecTree_D4_L3'] = [np.round(accuracy,2), np.round(precision, 2), np.round(recall, 2), np.round(f1score, 2)]

In [ ]:
coef_df

#### 2. Model k-nearest neighbors

Provem amb un model de K-nearest neighbors. En aquest model cal elegir el nombre de k nearest neighbors. S'elegeix inicialment 10.

In [ ]:
classifier = KNeighborsClassifier(n_neighbors=10)
classifier.fit(x_train, y_train)
unknown_points = x_test
y_pred = classifier.predict(unknown_points)

In [ ]:
x = 'Alcohol'
y = 'Hue'

d = {x:x_test[x], y:x_test[y], 'y_pred':y_pred}
plot_df = pd.DataFrame(data = d)

fig = plt.figure(dpi=100)
ax = fig.add_subplot()
lab_class = ['I', 'II', 'III']

colors = ['red', 'brown', 'black']
for i in range(1, 4):
    filter = plot_df['y_pred'] == i
    plot_class = plot_df[filter]
    ax.scatter(plot_class[x], plot_class[y],  c=colors[i-1], label=lab_class[i-1])
    plt.xlabel(x)
    plt.ylabel(y)
    
ax.legend()

Es pot comprovar amb un scatter plot com el model assigna cada mostra a una classe diferent. De moment, i si es compara **Hue** amb **Alcohol**, no sembla que aquest model sigui massa fiable.

In [ ]:
x = 'Flavanoids'
y = 'OD280/OD315'

d = {x:x_test[x], y:x_test[y], 'y_pred':y_pred}
plot_df = pd.DataFrame(data = d)

fig = plt.figure(dpi=100)
ax = fig.add_subplot()
lab_class = ['I', 'II', 'III']

colors = ['red', 'brown', 'black']
for i in range(1, 4):
    filter = plot_df['y_pred'] == i
    plot_class = plot_df[filter]
    ax.scatter(plot_class[x], plot_class[y],  c=colors[i-1], label=lab_class[i-1])
    plt.xlabel(x)
    plt.ylabel(y)
    
ax.legend()

També ho podem veure si enfronem **OD280/OD315** amb **Flavanoids**

In [ ]:
accuracy = accuracy_score(y_test, y_pred)
print(f'model accuracy: {np.round(accuracy, 2)}')

confusion_test = confusion_matrix(y_test, y_pred, labels=[1, 2, 3])

In [ ]:
plt.figure(figsize=(10, 6))
sns.set(font_scale=1.3)
plot_confusion_matrix(confusion_test, 
                      classes=['I','II', 'III'],
                      normalize= False,  
                      title='Matriu de confusió')

És evident que respecte amb el model anterior, aquest té un rendiment molt dolent. Fins a 18 mostres no están bé classificades.

In [ ]:
precision = precision_recall_fscore_support(y_test, y_pred, average='macro')[0]
recall = precision_recall_fscore_support(y_test, y_pred, average='macro')[1]
print(f'Precision: {np.round(precision, 2)}')
print(f'Recall: {np.round(recall, 2)}')
f1score = f1_score(y_test, y_pred, average='micro')
print(f'F1 score: {np.round(f1score, 2)}')

In [ ]:
coef_df['NK_O'] = [np.round(accuracy,2), np.round(precision, 2), np.round(recall, 2), np.round(f1score, 2)]

In [ ]:
coef_df

El rendiment es prou roí si es comparara amb l'arbre de decissió. En aquest cas, per a un model tipus k-nearest neighbors, convé normalitzar les dades d'entrada. Sinó, les variables amb magnituds d'ordre superiors afecten més en el resultat que els altres.

In [ ]:
mmscaler = MinMaxScaler()

for col in x_col:
    
    reshaped = np.array(wines[col]).reshape(-1,1)
    
    normalized = mmscaler.fit_transform(reshaped)
    
    if col == 'Alcohol':
        wines_norm = pd.DataFrame(normalized, columns=[col])
    else:
        wines_norm[col] = normalized
    
# Separem el dataframe en un conjunt per entrenar el model i un altre per aplicar-lo.
wines_train_n, wines_test_n = train_test_split(wines_norm, test_size=test_size_sel, random_state=42)

x_train_n = wines_train_n[x_col]
x_test_n = wines_test_n[x_col]

In [ ]:
x_train_n.describe() # Comprovem que es troben normalitzades les dades

In [ ]:
classifier = KNeighborsClassifier(n_neighbors=10)
classifier.fit(x_train_n, y_train)
unknown_points = x_test_n
y_pred = classifier.predict(unknown_points)

accuracy = accuracy_score(y_test, y_pred)
print(f'model accuracy: {np.round(accuracy, 2)}')

# confusion matrix

confusion_test = confusion_matrix(y_test, y_pred, labels=[1, 2, 3])

In [ ]:
plt.figure(figsize=(10, 6))
sns.set(font_scale=1.3)
plot_confusion_matrix(confusion_test, 
                      classes=['I','II', 'III'],
                      normalize= False,  
                      title='Matriu de confusió')

In [ ]:
x = 'Alcohol'
y = 'Hue'

d = {x:x_test[x], y:x_test[y], 'y_pred':y_pred}
plot_df = pd.DataFrame(data = d)

fig = plt.figure(dpi=100)
ax = fig.add_subplot()
lab_class = ['I', 'II', 'III']

colors = ['red', 'brown', 'black']
for i in range(1, 4):
    filter = plot_df['y_pred'] == i
    plot_class = plot_df[filter]
    ax.scatter(plot_class[x], plot_class[y],  c=colors[i-1], label=lab_class[i-1])
    plt.xlabel(x)
    plt.ylabel(y)
    
ax.legend()

In [ ]:
precision = precision_recall_fscore_support(y_test, y_pred, average='macro')[0]
recall = precision_recall_fscore_support(y_test, y_pred, average='macro')[1]
print(f'Precision: {np.round(precision, 2)}')
print(f'Recall: {np.round(recall, 2)}')
f1score = f1_score(y_test, y_pred, average='micro')
print(f'F1 score: {np.round(f1score, 2)}')

In [ ]:
coef_df['NK_Normalized'] = [np.round(accuracy,2), np.round(precision, 2), np.round(recall, 2), np.round(f1score, 2)]

In [ ]:
coef_df

El rendiment millora de manera molt notable.

En el model KN podem variar el número k. A mesura que utilitzem un número menor, podem incrementar la precisió del model però podem acabar sobreajustant el model. Iterarem augmentant de manera progresiva per veure com reacciona el model als diferents valors de k.

In [ ]:
for k in range(1, 10):
    classifier = KNeighborsClassifier(n_neighbors=k)
    classifier.fit(x_train_n, y_train)
    unknown_points = x_test_n
    y_pred = classifier.predict(unknown_points)
    accuracy = accuracy_score(y_test, y_pred)
    print(f'n_neighbors:{k}')
    print(f'accuracy {np.round(accuracy, 3)}')
    scores = cross_validate(classifier, x_train_n, y_train, cv=5, scoring='accuracy', return_train_score=True)
    print(f'Entrenament: {scores["train_score"]}')
    print(f'Test       : {scores["test_score"]}')
    print('-------------------')

En aquest cas, les diferències entre les diferents classes son tan notables que la precisió quasi no varia. Convé  elegir un k superior a 1, ja que el model agafa de referència sols el valor més proper per a categoritzar una mostra i es produeix overfitting.

In [ ]:
k = 5

classifier = KNeighborsClassifier(n_neighbors=k)
classifier.fit(x_train_n, y_train)
unknown_points = x_test_n
y_pred = classifier.predict(unknown_points)
accuracy = accuracy_score(y_test, y_pred)

precision = precision_recall_fscore_support(y_test, y_pred, average='macro')[0]
recall = precision_recall_fscore_support(y_test, y_pred, average='macro')[1]
f1score = f1_score(y_test, y_pred, average='micro')


coef_df['NK_Normalized_k5'] = [np.round(accuracy,2), np.round(precision, 2), np.round(recall, 2), np.round(f1score, 2)]

#### 3. Random Forest

Finalment, de manera similar a com feiem amb l'arbre de decisió. Podem provar un model una mica més complexe amb el Random Forest.

In [ ]:
from sklearn.ensemble import RandomForestClassifier

rfc = RandomForestClassifier(random_state=42)
rfc.fit(x_train, y_train)
y_pred = rfc.predict(x_test)

In [ ]:
accuracy = accuracy_score(y_test, y_pred)
print(f'model accuracy: {np.round(accuracy,2)}')



In [ ]:
confusion_test = confusion_matrix(y_test, y_pred, labels=[1, 2, 3])
plt.figure(figsize=(5, 5))
sns.set(font_scale=1.3)
plot_confusion_matrix(confusion_test, 
                      classes=['I','II', 'III'],
                      normalize= False,  
                      title='Matriu de confusió')

In [ ]:
precision = precision_recall_fscore_support(y_test, y_pred, average='macro')[0]
recall = precision_recall_fscore_support(y_test, y_pred, average='macro')[1]
print(f'Precision: {np.round(precision, 2)}')
print(f'Recall: {np.round(recall, 2)}')
f1score = f1_score(y_test, y_pred, average='micro')
print(f'F1 score: {np.round(f1score, 2)}')

El rendiment és el millor que hem pogut obtindre.

En Random Forest també podem elegir un **max_depth** determinat. Anem a provar amb diferents valors i estudiar la seua precissió

In [ ]:
for k in range(1, 9):

    rfc = RandomForestClassifier(max_depth=k, random_state=42)
    rfc.fit(x_train, y_train)
    y_pred = rfc.predict(x_test)
    accuracy = accuracy_score(y_test, y_pred)
    print(f'max_depth: {k}')
    print(f'model accuracy: {np.round(accuracy,2)}')
    scores = cross_validate(rfc, x_train, y_train, cv=5, scoring='accuracy', return_train_score=True)
    print(f'Entrenament: {scores["train_score"]}')
    print(f'Test       : {scores["test_score"]}')
    print('-------------------')

Obtenim una precisió molt bona pràcticament amb qualsevol k però el model pateix d'overfitting si el max_depth supera el 3. Així que ens quedem amb eixe valor.

In [ ]:
rfc = RandomForestClassifier(max_depth=3, random_state=42)
rfc.fit(x_train, y_train)
y_pred = rfc.predict(x_test)
accuracy = accuracy_score(y_test, y_pred)

In [ ]:
x = 'Alcohol'
y = 'Hue'

d = {x:x_test[x], y:x_test[y], 'y_pred':y_pred}
plot_df = pd.DataFrame(data = d)

fig = plt.figure(dpi=100)
ax = fig.add_subplot()
lab_class = ['I', 'II', 'III']

colors = ['red', 'brown', 'black']
for i in range(1, 4):
    filter = plot_df['y_pred'] == i
    plot_class = plot_df[filter]
    ax.scatter(plot_class[x], plot_class[y],  c=colors[i-1], label=lab_class[i-1])
    plt.xlabel(x)
    plt.ylabel(y)
    
ax.legend()

In [ ]:
x = 'OD280/OD315'
y = 'Hue'

d = {x:x_test[x], y:x_test[y], 'y_pred':y_pred}
plot_df = pd.DataFrame(data = d)

fig = plt.figure(dpi=100)
ax = fig.add_subplot()
lab_class = ['I', 'II', 'III']

colors = ['red', 'brown', 'black']
for i in range(1, 4):
    filter = plot_df['y_pred'] == i
    plot_class = plot_df[filter]
    ax.scatter(plot_class[x], plot_class[y],  c=colors[i-1], label=lab_class[i-1])
    plt.xlabel(x)
    plt.ylabel(y)
    
ax.legend()

In [ ]:
precision = precision_recall_fscore_support(y_test, y_pred, average='macro')[0]
recall = precision_recall_fscore_support(y_test, y_pred, average='macro')[1]
f1score = f1_score(y_test, y_pred, average='micro')

In [ ]:
coef_df['RF_5'] = [np.round(accuracy,2), np.round(precision, 2), np.round(recall, 2), np.round(f1score, 2)]

In [ ]:
coef_df

Vegem com el model que millor rendeix és, sense dubte, el Random Forest. Mentre que els altres dos, una vegada elegits bé els paràmetres, mostren un rendiment similar.